<a href="https://colab.research.google.com/github/kapibara3chi/NLPTesForGC/blob/main/NLPTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pip

In [2]:
!pip  install -U ginza ja_ginza_electra

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.8 MB 22.8 MB/s 
     |████████████████████████████████| 2.2 MB 43.7 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |████████████████████████████████| 283 kB 59.9 MB/s 
     |████████████████████████████████| 1.1 MB 55.0 MB/s 
     |████████████████████████████████| 4.7 MB 46.9 MB/s 
     |████████████████████████████████| 7.6 MB 48.9 MB/s 
     |████████████████████████████████| 182 kB 48.8 MB/s 
     |████████████████████████████████| 6.6 MB 41.2 MB/s 
  Created wheel for SudachiDict-core: filename=SudachiDict_core-20221021-py3-none-any.whl size=71574782 sha256=097ad2727be7338febccc55c4dfb98f181daa0ca8159a6f9ddc0279f7763604b
  Stored in directory: /root/.cache/pip/wheels/ca/e3/ed/e78fecf6fd34349114d292242a16fc08d513fb32c2d9c5d786
  Created wheel for sudachitra: filename=SudachiTra-0.1.7-py3-none-any.whl size=266095 sha256=bd24

# main

In [3]:
import ginza
from spacy import displacy
import spacy
class DependencyAnalysis:
    def __init__(self):
        self.nlp = spacy.load('ja_ginza_electra')

    def run(self, text):
        """係り受け解析"""
        doc = self.nlp(text)

        token_head_list = []

        for sent in doc.sents:
            for token in sent:
                token_head_list.append(
                    {"i": token.i, "orth": token.orth_, "base": token.lemma_, "head": token.head.i, "dep": token.dep_})

        displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})

        return token_head_list
if __name__ == '__main__':
  dependency = DependencyAnalysis()
  ret=dependency.run("下流あごをだすと分離は改善するが、コンタミが悪化する")
  print(ret)

error megagonlabs/transformers-ud-japanese-electra-base-ginza-510 does not appear to have a file named config.json.
error We couldn't connect to 'https://huggingface.co' to load this model, couldn't find it in the cached files and it looks like /tmp/tmpx59uhpa2/config.json is not the path to a directory containing a config.json file.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.
trying to download model from huggingface hub: megagonlabs/transformers-ud-japanese-electra-base-ginza-510 ...


succeded
/usr/local/lib/python3.8/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


[{'i': 0, 'orth': '下流', 'base': '下流', 'head': 1, 'dep': 'compound'}, {'i': 1, 'orth': 'あご', 'base': 'あご', 'head': 3, 'dep': 'obj'}, {'i': 2, 'orth': 'を', 'base': 'を', 'head': 1, 'dep': 'case'}, {'i': 3, 'orth': 'だす', 'base': 'だす', 'head': 7, 'dep': 'advcl'}, {'i': 4, 'orth': 'と', 'base': 'と', 'head': 3, 'dep': 'mark'}, {'i': 5, 'orth': '分離', 'base': '分離', 'head': 7, 'dep': 'nsubj'}, {'i': 6, 'orth': 'は', 'base': 'は', 'head': 5, 'dep': 'case'}, {'i': 7, 'orth': '改善', 'base': '改善', 'head': 13, 'dep': 'advcl'}, {'i': 8, 'orth': 'する', 'base': 'する', 'head': 7, 'dep': 'aux'}, {'i': 9, 'orth': 'が', 'base': 'が', 'head': 7, 'dep': 'mark'}, {'i': 10, 'orth': '、', 'base': '、', 'head': 7, 'dep': 'punct'}, {'i': 11, 'orth': 'コンタミ', 'base': 'コンタミ', 'head': 13, 'dep': 'nsubj'}, {'i': 12, 'orth': 'が', 'base': 'が', 'head': 11, 'dep': 'case'}, {'i': 13, 'orth': '悪化', 'base': '悪化', 'head': 13, 'dep': 'ROOT'}, {'i': 14, 'orth': 'する', 'base': 'する', 'head': 13, 'dep': 'aux'}]
